# Convolutional Neural Network

### Importing the libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [5]:
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8002 images belonging to 2 classes.


In [6]:
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2021 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
cnn = Sequential()

### Step 1 - Convolution

In [8]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [9]:
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [10]:
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [13]:
cnn.add(Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=50,
    validation_data=test_set,
    validation_steps=len(test_set)
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 1611s 6s/step - accuracy: 0.5536 - loss: 0.6904 - val_accuracy: 0.7031 - val_loss: 0.5937
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


251/251 ━━━━━━━━━━━━━━━━━━━━ 147s 452ms/step - accuracy: 0.6695 - loss: 0.6126 - val_accuracy: 0.7105 - val_loss: 0.5865
Epoch 4/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 117s 461ms/step - accuracy: 0.7088 - loss: 0.5699 - val_accuracy: 0.7496 - val_loss: 0.5205
Epoch 6/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 133us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
 85/251 ━━━━━━━━━━━━━━━━━━━━ 1:05 395ms/step - accuracy: 0.7185 - loss: 0.5279

In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy:.2f}")

## Part 4 - Making a single prediction

In [ ]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale

    prediction = cnn.predict(img_array)
    if prediction[0][0] > 0.5:
        print("Prediction: Dog")
    else:
        print("Prediction: Cat")

In [ ]:
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg')
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_2.jpg')

In [ ]:
cnn.save('cat_dog_classifier.h5')

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('cat_dog_classifier.h5')